In [1]:
# Take a csv file that has signals data and basically sample from it a bunch of datapoints however we want
# Could be in increments of time intervals 
# Basically say in the past time
# Take a activity dataset and basically sample however we want from it

# Load Signal and Activity Dataset

In [143]:
import pandas as pd

BASE_NUM = "05"
SIGNAL_PATH = f"./datasets/processed_sig{BASE_NUM}.txt"
ACTIVITY_PATH = f"./datasets/processed_act{BASE_NUM}.txt"

def load_signal_csv(csv_path):
    df = pd.read_csv(csv_path)
    # Remove columns that have invalid data
    clean_df = df.loc[:, ~df.columns.isin(['rssnr', 'cqi','ta'])]
    return clean_df

signal_df = load_signal_csv(SIGNAL_PATH)


def load_activity_csv(path):
    # This dataframe contains time, activity, status(enter/exit)
    activity_df = pd.read_csv(path)
    # Ignore EXIT state since we only care about current state
    activity_df = activity_df[activity_df['status'] != "EXIT"]
    return activity_df

activity_df = load_activity_csv(ACTIVITY_PATH)

In [60]:
signal_df.head()


,time,mRegistered,mTimeStamp,mPci,mTac,mEarfcn,mMcc+mMnc,ss,rsrp,rsrq
0,11-07 16:52:51.012,YES,22841626089569,116,16185,2300,310260,20,-103,-8
1,11-07 16:52:51.012,NO,22841626089569,99,2147483647,2300,nullnull,12,-116,-17
2,11-07 16:52:51.012,NO,22841626089569,116,2147483647,675,nullnull,11,-108,-8
3,11-07 16:52:51.012,NO,22841626089569,44,2147483647,675,nullnull,11,-117,-16
4,11-07 16:52:51.270,NO,22841626089569,44,2147483647,675,nullnull,11,-117,-16


In [61]:
activity_df.head()

,time,activity,status
0,11-07 16:50:26.886,STILL,ENTER
1,11-07 16:50:26.895,STILL,ENTER
2,11-07 16:52:57.316,STILL,ENTER
3,11-07 16:52:57.331,STILL,ENTER
4,11-07 16:52:57.337,STILL,ENTER


# Merge Datasets

In [62]:
def search_activity_status_at_time(query_time, activity_df):

    # Searches the activity_df to get the
    # state of the device at time
    
    no_value_string = "NOVALUE"
    last_observed_state = no_value_string
    activity_time_passed_query_time = False
    
    for _,row in activity_df.iterrows():
        if query_time < row['time']:
            # Passed the state we wanted
            activity_time_passed_query_time = True
            break
        
        last_observed_state = row['activity']
    
    if not activity_time_passed_query_time:
        # time is sometime in the future after acitivity data
        last_observed_state = no_value_string
    
    return last_observed_state

# Example of searching for activity at specific time
# search_activity_status_at_time("11-06 16:50:26.900", activity_df)
# search_activity_status_at_time("11-07 16:50:26.900", activity_df)
def create_merged_dataset(signal_df, activity_df):
    merged_df = signal_df.copy()
    merged_df['activity'] = "INVALID"
    cached_result = {}
    for i,row, in signal_df.iterrows():
        time = row['time']
        if time not in cached_result:
            cached_result[time] = search_activity_status_at_time(row['time'], activity_df)
        merged_df.loc[i,'activity'] = cached_result[time]
    return merged_df

In [63]:
merged_df = create_merged_dataset(signal_df, activity_df)
merged_df.head()

,time,mRegistered,mTimeStamp,mPci,mTac,mEarfcn,mMcc+mMnc,ss,rsrp,rsrq,activity
0,11-07 16:52:51.012,YES,22841626089569,116,16185,2300,310260,20,-103,-8,STILL
1,11-07 16:52:51.012,NO,22841626089569,99,2147483647,2300,nullnull,12,-116,-17,STILL
2,11-07 16:52:51.012,NO,22841626089569,116,2147483647,675,nullnull,11,-108,-8,STILL
3,11-07 16:52:51.012,NO,22841626089569,44,2147483647,675,nullnull,11,-117,-16,STILL
4,11-07 16:52:51.270,NO,22841626089569,44,2147483647,675,nullnull,11,-117,-16,STILL


# Drop Duplicates Using mTimeStamp and mPci

Now that we have merged the two datasets lets drop any duplicates identified using the mTimestamp


In [135]:
length_before = len(merged_df)
df_no_duplicates = merged_df.drop_duplicates(subset=["mTimeStamp", "mPci"]).reset_index(drop=True)
print(f"Trimmed from {length_before} to {len(df_no_duplicates)}")

Trimmed from 50321 to 7645


# Mark Points with Handover 

In [136]:
def get_max_at_time(df, time):
    timestep_df = df[df['time'] == time].reset_index(drop=True)
    timestep_df = timestep_df[timestep_df['mRegistered'] == "NO"]

    if len(timestep_df) == 0:
        return {
            "max_ss": 0,
            "max_rsrp": -1000,
            "max_rsrq": -1000,
        }

    ret = {
        "max_ss": max(timestep_df['ss'].values),
        "max_rsrp": max(timestep_df['rsrp'].values),
        "max_rsrq": max(timestep_df['rsrq'].values),
    }
    return ret

merged_registered_df = df_no_duplicates[df_no_duplicates['mRegistered'] == "YES"].reset_index(drop=True)
merged_registered_df['nextAP'] = merged_registered_df['mPci'].shift(-1)
merged_registered_df = merged_registered_df[:-1]
def label_handover_occured(row):
    if row['mPci'] == row['nextAP']:
        return 0
    return 1
merged_registered_df['handoverOccured'] = merged_registered_df.apply(lambda row: label_handover_occured(row), axis=1)
merged_registered_df['max_ss'] = merged_registered_df.apply(lambda row: get_max_at_time(df_no_duplicates, row['time'])['max_ss'], axis=1)
merged_registered_df['max_rsrp'] = merged_registered_df.apply(lambda row: get_max_at_time(df_no_duplicates, row['time'])['max_rsrp'], axis=1)
merged_registered_df['max_rsrq'] = merged_registered_df.apply(lambda row: get_max_at_time(df_no_duplicates, row['time'])['max_rsrq'], axis=1)

print(merged_registered_df['handoverOccured'].value_counts())
merged_registered_df.head()

0    1957
1     103
Name: handoverOccured, dtype: int64


,time,mRegistered,mTimeStamp,mPci,mTac,mEarfcn,mMcc+mMnc,ss,rsrp,rsrq,activity,nextAP,handoverOccured,max_ss,max_rsrp,max_rsrq
0,11-07 16:52:51.012,YES,22841626089569,116,16185,2300,310260,20,-103,-8,STILL,116.0,0,12,-116,-16
1,11-07 16:52:53.123,YES,22843741769624,116,16185,2300,310260,17,-108,-9,STILL,116.0,0,11,-117,-16
2,11-07 16:52:55.621,YES,22846239216799,116,16185,2300,310260,20,-103,-7,STILL,116.0,0,13,-115,-16
3,11-07 16:52:58.111,YES,22848459999104,116,16185,2300,310260,22,-104,-14,STILL,116.0,0,15,-112,-16
4,11-07 16:53:00.119,YES,22850739235008,116,16185,2300,310260,19,-105,-8,STILL,116.0,0,14,-116,-18


In [137]:
# Add secondary features showing difference between selected AP and other APs
merged_registered_df['diff_in_ss'] = merged_registered_df['ss'] - merged_registered_df['max_ss']
merged_registered_df['diff_in_rsrp'] = merged_registered_df['rsrp'] - merged_registered_df['max_rsrp']
merged_registered_df['diff_in_rsrq'] = merged_registered_df['rsrq'] - merged_registered_df['max_rsrq']


In [140]:
merged_registered_df[merged_registered_df['handoverOccured'] == 1]

,time,mRegistered,mTimeStamp,mPci,mTac,mEarfcn,mMcc+mMnc,ss,rsrp,rsrq,activity,nextAP,handoverOccured,max_ss,max_rsrp,max_rsrq,diff_in_ss,diff_in_rsrp,diff_in_rsrq
70,11-07 16:57:39.656,YES,22967932709391,116,16185,675,310260,18,-112,-16,WALKING,99.0,1,31,0,0,-13,-112,-16
119,11-07 17:02:03.581,YES,23027665525550,99,16185,2300,310260,17,-117,-17,STILL,44.0,1,14,-114,-16,3,-3,-1
120,11-07 17:02:09.652,YES,23028050691995,44,16185,675,310260,19,-112,-18,STILL,99.0,1,12,-112,-14,7,0,-4
122,11-07 17:02:17.813,YES,23030549233389,99,16185,675,310260,18,-111,-15,WALKING,116.0,1,18,-108,-16,0,-3,1
133,11-07 17:02:50.840,YES,23054326562789,116,16185,2300,310260,23,-103,-16,WALKING,46.0,1,16,-112,-20,7,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,11-07 19:43:38.279,YES,25486237611513,219,34624,850,310410,30,-91,-15,UNKNOWN,289.0,1,26,-85,-12,4,-6,-3
2018,11-07 19:44:08.325,YES,25488860094952,289,34624,9820,310410,22,-104,-16,UNKNOWN,81.0,1,28,-79,-9,-6,-25,-7
2023,11-07 19:44:34.318,YES,25494268333459,81,34624,850,310410,28,-94,-15,UNKNOWN,251.0,1,27,-75,-8,1,-19,-7
2047,11-07 19:46:47.263,YES,25519712269379,251,34624,9820,310410,25,-100,-19,UNKNOWN,243.0,1,26,-80,-7,-1,-20,-12


In [141]:
merged_registered_df.loc[65:75]

,time,mRegistered,mTimeStamp,mPci,mTac,mEarfcn,mMcc+mMnc,ss,rsrp,rsrq,activity,nextAP,handoverOccured,max_ss,max_rsrp,max_rsrq,diff_in_ss,diff_in_rsrp,diff_in_rsrq
65,11-07 16:57:21.668,YES,22960050597616,116,16185,675,310260,19,-107,-11,WALKING,116.0,0,31,0,0,-12,-107,-11
66,11-07 16:57:24.945,YES,22961743350233,116,16185,675,310260,16,-112,-10,WALKING,116.0,0,31,0,0,-15,-112,-10
67,11-07 16:57:27.556,YES,22963839913931,116,16185,675,310260,20,-110,-17,WALKING,116.0,0,31,0,0,-11,-110,-17
68,11-07 16:57:29.132,YES,22965897115699,116,16185,675,310260,19,-111,-15,WALKING,116.0,0,31,0,0,-12,-111,-15
69,11-07 16:57:33.615,YES,22967026833728,116,16185,675,310260,18,-112,-12,WALKING,116.0,0,31,0,0,-13,-112,-12
70,11-07 16:57:39.656,YES,22967932709391,116,16185,675,310260,18,-112,-16,WALKING,99.0,1,31,0,0,-13,-112,-16
71,11-07 16:57:45.606,YES,22968729369731,99,16185,2300,310260,19,-110,-13,WALKING,99.0,0,31,0,0,-12,-110,-13
72,11-07 16:57:45.698,YES,22968758530463,99,16185,2300,310260,19,-110,-13,WALKING,99.0,0,31,0,0,-12,-110,-13
73,11-07 16:57:51.589,YES,22969506483194,99,16185,2300,310260,19,-107,-12,WALKING,99.0,0,31,0,0,-12,-107,-12
74,11-07 16:58:03.560,YES,22970207803680,99,16185,2300,310260,23,-103,-14,WALKING,99.0,0,31,0,0,-8,-103,-14


In [146]:
merged_registered_df.to_csv(f"./datasets/merged{BASE_NUM}.txt", index=False)